In [39]:
import os
import csv
from bs4 import BeautifulSoup
import requests
import time
import datetime
import smtplib
import pandas as pd
import re
import unicodedata
from selenium import webdriver


In [45]:
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Driver [C:\Users\TuanA\.wdm\drivers\chromedriver\win32\104.0.5112.79\chromedriver.exe] found in cache
C:\Users\TuanA\AppData\Local\Temp\ipykernel_18816\2518127973.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [46]:
URL = "https://www.amazon.ca"
driver.get(URL)

In [36]:
def get_url(search_term):
    """Generate a url from search term"""
    template = "https://www.amazon.ca/s?k={}&crid=KQWQ1T01TXQ2&sprefix=office+desk%2Caps%2C79&ref=nb_sb_noss_1"
    search_term = search_term.replace(" ", '+')

    #add term querry to url
    url = template.format(search_term)
    #add page querry placeholder
    url += '&page{}'
    return url

In [7]:
url = get_url('ultrawide monitor')
print(url)

https://www.amazon.ca/s?k=ultrawide+monitor&crid=KQWQ1T01TXQ2&sprefix=office+desk%2Caps%2C79&ref=nb_sb_noss_1


In [8]:
driver.get(url)

<h2>Extract the collection</h2>

In [9]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [10]:
results = soup.find_all('div', {'data-component-type': 's-search-result'})

In [11]:
len(results)

60

## Prototype the record ##

In [12]:
item = results[0]

In [13]:
atag = item.h2.a

In [15]:
description = atag.text.strip()

In [17]:
url = 'https://www.amazon.ca' + atag.get('href')

In [18]:
price_parent = item.find('span', 'a-price')

In [19]:
price = price_parent.find('span', 'a-offscreen').text

In [22]:
rating = item.i.text

In [28]:
review_count = item.find('span', {'class': 'a-size-base s-underline-text'}).text

## Generalize the pattern ##

In [32]:
def extract_record(item):
    """Extract and return data from a single record"""

    #description and url 
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.ca' + atag.get('href')

    try:
        #price
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return
    try:
        #rating
        rating = item.i.text
        review_count = item.find('span', {'class': 'a-size-base s-underline-text'}).text
    except AttributeError:
        rating = ""
        review_count = ''

    result = (description, price, rating, review_count, url)
    return result

In [33]:
records = []
results = soup.find_all('div', {'data-component-type': 's-search-result'})

for item in results:
    record = extract_record(item)
    if record:
        records.append(extract_record(item))

In [34]:
records[0]

('SAMSUNG 32 Inch 4K UHD Monitor, Computer Monitor, Wide Monitor, HDMI Monitor, HDR 10 (1 Billion Colors), 3 Sided Borderless Design, TUV-Certified Intelligent Eye Care, S70A (LS32A704NWNXZA)',
 '$398.00',
 '4.1 out of 5 stars',
 '1,282',
 'https://www.amazon.ca/sspa/click?ie=UTF8&spc=MTo4MDQ2Njk5MTI3NzY4MDE0OjE2NjEwMDM5Mjg6c3BfYXRmOjIwMDA1NDU1NDY4MzQ5ODo6MDo6&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGY&url=%2FComputer-Borderless-TUV-Certified-Intelligent-LS32A704NWNXZA%2Fdp%2FB08YFM2F67%2Fref%3Dsr_1_1_sspa%3Fcrid%3DKQWQ1T01TXQ2%26keywords%3Dultrawide%2Bmonitor%26qid%3D1661003928%26sprefix%3Doffice%2Bdesk%252Caps%252C79%26sr%3D8-1-spons%26psc%3D1')

In [49]:
def main(search_term):
    """Run main program routine"""
    driver = webdriver.Chrome(ChromeDriverManager().install())

    record = []
    url = get_url(search_term)

    for page in range(1, 21):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div', {'data-component-type': 's-search-result'})

        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
    driver.close()

    # save data to csv file
    with open('result.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description', 'Price', 'Rating', 'ReviewCount', 'url'])
        writer.writerows(records)

    

In [50]:
main('ultrawide monitor')

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Driver [C:\Users\TuanA\.wdm\drivers\chromedriver\win32\104.0.5112.79\chromedriver.exe] found in cache
C:\Users\TuanA\AppData\Local\Temp\ipykernel_18816\3865522643.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
